In [108]:
import numpy as np
import pandas as pd
import gc

In [2]:
attr = pd.read_csv('/kaggle/input/vk-case/attr.csv')

In [81]:
def friendship(sampled_data):

    sampled_data['u_w'] = ''
    sampled_data['v_w'] = ''
    from tqdm import tqdm

    ego_list = sampled_data['ego_id'].unique()
    sampled_data['u_w'] = sampled_data['u_w'].astype(object)
    sampled_data['v_w'] = sampled_data['v_w'].astype(object)

    for ego in tqdm(ego_list):
        temp = sampled_data[sampled_data["ego_id"] == ego]
        for i, (u, v) in enumerate(zip(temp["u"], temp["v"])):
            u_w1 = temp.loc[temp["v"] == u, "u"].dropna().values.tolist()
            u_w2 = temp.loc[temp["u"] == u, "v"].dropna().values.tolist()
            u_w = u_w1 + u_w2
            v_w1 = temp.loc[temp["u"] == v, "v"].dropna().values.tolist()
            v_w2 = temp.loc[temp["v"] == v, "u"].dropna().values.tolist()
            v_w = v_w1 + v_w2
            sampled_data.at[temp.index[i], 'u_w'] = u_w
            sampled_data.at[temp.index[i], 'v_w'] = v_w
    return sampled_data

In [84]:
def jaccard(u_w, v_w):
    intersection = len(list(set(u_w).intersection(v_w)))
    union = (len(u_w) + len(v_w)) - intersection
    return float(intersection) / union

In [ ]:
from IPython.display import FileLink
sampled_data.to_csv('/kaggle/working/sampled_data1.csv', index = False)
FileLink(r'sampled_data1.csv')

In [79]:
def dif_school(row):
    if row['school_u'] not in (-1, 0) and row['school_v'] not in (-1, 0):
        val = np.abs(row['school_u'] - row['school_v'])
    if row['school_u'] in (-1, 0) and row['school_v'] in (-1, 0):
        val = -1
    if row['school_u'] in (-1, 0) and row['school_v'] not in (-1, 0):
        val = row['school_v']
    if row['school_u'] not in (-1, 0) and row['school_v'] in (-1, 0):
        val = row['school_u']
    return val

def dif_city(row):
    if row['city_id_u'] not in (-1, 0) and row['city_id_v'] not in (-1, 0):
        val = np.abs(row['city_id_u'] - row['city_id_v'])
    if row['city_id_u'] in (-1, 0) and row['city_id_v'] in (-1, 0):
        val = -1
    if row['city_id_u'] in (-1, 0)and row['city_id_v'] not in (-1, 0):
        val = row['city_id_v']
    if row['city_id_u'] not in (-1, 0) and row['city_id_v'] in (-1, 0):
        val = row['city_id_u']
    return val


def dif_uni(row):
    if row['university_u'] not in (-1, 0) and row['university_v'] not in (-1, 0):
        val = np.abs(row['university_u'] - row['university_v'])
    if row['university_u'] in (-1, 0) and row['university_v'] in (-1, 0):
        val = -1
    if row['university_u'] in (-1, 0) and row['university_v'] not in (-1, 0):
        val = row['university_v']
    if row['university_u'] not in (-1, 0) and row['university_v'] in (-1, 0):
        val = row['university_u']
    return val

In [75]:
def replace_with_mean(value):
    if (value > 70).any():
        return value.mean()
    else:
        return value

In [ ]:
test1 = test1.merge(attr, on=['ego_id', 'u'], how='left')
test1.rename(columns = {'age':' age_u', 'city_id':'city_id_u', 'sex':'sex_u',
                        'school':'school_u','university':'university_u',}, inplace = True )
test1 = test1.merge(attr, left_on=['ego_id', 'v'], right_on=['ego_id', 'u'], how='left')
test1.rename(columns = {'age':'age_v', 'city_id':'city_id_v', 'sex':'sex_v',
                        'school':'school_v','university':'university_v',}, inplace = True )
test1.rename(columns = {'u_x':'u'}, inplace = True )

test1 = test1.drop(columns = ['u_y'])

In [384]:
test1.duplicated()

0           False
1           False
2           False
3           False
4           False
            ...  
40548775    False
40548776    False
40548777    False
40548778    False
40548779    False
Length: 40548780, dtype: bool

In [388]:
test1 = test1[test1['x1'].isna()]

In [389]:
test1[["ego_id", "u", "v"]].equals(submission[["ego_id", "u", "v"]])

False

In [391]:
submission1 = pd.read_csv('/kaggle/input/vk-case/submission.csv')

In [390]:
submission1.merge(test1, on=['ego_id', 'u', 'v'], how='inner')

9           False
10          False
20          False
26          False
28          False
            ...  
40548756    False
40548757    False
40548762    False
40548767    False
40548772    False
Length: 8109756, dtype: bool

_________________________________________

In [60]:
submission = pd.read_csv('/kaggle/input/vk-case/submission.csv')
test = pd.read_csv('/kaggle/input/vk-case/test.csv')
submission.shape

(810976, 4)

In [61]:
subm = submission[['ego_id', 'u', 'v']]
subm

,ego_id,u,v
0,8,0,93
1,8,0,143
2,8,0,151
3,8,1,24
4,8,5,4
...,...,...,...
810971,1709396984676,73,23
810972,1709396984676,74,68
810973,1709396984676,77,28
810974,1709396984676,79,38


In [63]:
test['t'] = test['t'].fillna(-1)

In [19]:
test.isna().sum()

ego_id          0
u               0
v               0
t               0
x1        6861443
x2              0
x3              0
dtype: int64

In [64]:
subm = subm.merge(test, on=['ego_id', 'u', 'v'], how='inner')

In [65]:
subm = subm[subm['x1'].isna()]

In [66]:
subm.isna().sum()

ego_id         0
u              0
v              0
t              0
x1        810983
x2             0
x3             0
dtype: int64

In [56]:
subm

,ego_id,u,v,t,x1,x2,x3
0,8,0,93,359.6,NaN,0.000000,0.0
1,8,0,143,6.1,NaN,0.000000,0.0
2,8,0,151,0.2,NaN,1.386294,0.0
3,8,1,24,594.5,NaN,0.000000,0.0
4,8,5,4,461.5,NaN,0.000000,0.0
...,...,...,...,...,...,...,...
811028,1709396984676,73,23,20.3,NaN,0.000000,0.0
811029,1709396984676,74,68,46.7,NaN,0.000000,0.0
811030,1709396984676,77,28,43.4,NaN,0.000000,0.0
811031,1709396984676,79,38,50.2,NaN,0.000000,0.0


In [67]:
subm = subm.merge(attr, on=['ego_id', 'u'], how='left')
subm.rename(columns = {'age':' age_u', 'city_id':'city_id_u', 'sex':'sex_u',
                        'school':'school_u','university':'university_u',}, inplace = True )
subm = subm.merge(attr, left_on=['ego_id', 'v'], right_on=['ego_id', 'u'], how='left')
subm.rename(columns = {'age':'age_v', 'city_id':'city_id_v', 'sex':'sex_v',
                        'school':'school_v','university':'university_v',}, inplace = True )
subm.rename(columns = {'u_x':'u'}, inplace = True )

subm = subm.drop(columns = ['u_y'])

In [72]:
subm

,ego_id,u,v,t,x1,x2,x3,age_u,city_id_u,sex_u,school_u,university_u,age_v,city_id_v,sex_v,school_v,university_v
0,8,0,93,359.6,0.0,0.000000,0.0,36.0,979281502.0,2.0,213987831.0,845825535.0,36.0,979281502.0,2.0,734952557.0,566091832.0
1,8,0,143,6.1,0.0,0.000000,0.0,36.0,979281502.0,2.0,213987831.0,845825535.0,43.0,979281502.0,2.0,-1.0,-1.0
2,8,0,151,0.2,0.0,1.386294,0.0,36.0,979281502.0,2.0,213987831.0,845825535.0,18.0,979281502.0,2.0,-1.0,-1.0
3,8,1,24,594.5,0.0,0.000000,0.0,120.0,56833659.0,1.0,370230497.0,779615128.0,36.0,104874069.0,2.0,213987831.0,562436811.0
4,8,5,4,461.5,0.0,0.000000,0.0,37.0,979281502.0,1.0,814552332.0,-1.0,37.0,-1.0,1.0,213987831.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810978,1709396984676,73,23,20.3,0.0,0.000000,0.0,122.0,-1.0,1.0,405077490.0,-1.0,18.0,-1.0,1.0,-1.0,-1.0
810979,1709396984676,74,68,46.7,0.0,0.000000,0.0,20.0,-1.0,2.0,-1.0,-1.0,17.0,338248086.0,2.0,485661706.0,-1.0
810980,1709396984676,77,28,43.4,0.0,0.000000,0.0,16.0,926522633.0,1.0,43512792.0,112540362.0,14.0,338248086.0,1.0,253082810.0,-1.0
810981,1709396984676,79,38,50.2,0.0,0.000000,0.0,16.0,269576388.0,2.0,566558003.0,-1.0,112.0,-1.0,1.0,253082810.0,-1.0


In [73]:
subm['t'] = subm['t'].fillna(-1)
subm = subm.fillna(0)

In [77]:
subm['submage_u'] = subm.groupby('ego_id')[' age_u'].transform(replace_with_mean)
subm['age_v'] = subm.groupby('ego_id')['age_v'].transform(replace_with_mean)

In [80]:
subm['dif_school'] =subm.apply(dif_school, axis=1)
subm['dif_city'] = subm.apply(dif_city, axis=1)
subm['dif_uni'] = subm.apply(dif_uni, axis=1)

In [82]:
%%time

subm = friendship(subm)

100%|██████████| 20586/20586 [19:22<00:00, 17.72it/s] 

CPU times: user 19min 18s, sys: 15.8 s, total: 19min 34s
Wall time: 19min 22s


In [85]:
subm['jac'] = subm.apply(lambda row: jaccard(row['u_w'], row['v_w']), axis=1)

In [86]:
subm

,ego_id,u,v,t,x1,x2,x3,age_u,city_id_u,sex_u,...,sex_v,school_v,university_v,submage_u,dif_school,dif_city,dif_uni,u_w,v_w,jac
0,8,0,93,359.6,0.0,0.000000,0.0,36.0,979281502.0,2.0,...,2.0,734952557.0,566091832.0,52.550000,520964726.0,0.0,279733703.0,"[11, 55, 75, 93, 143, 151]",[0],0.0
1,8,0,143,6.1,0.0,0.000000,0.0,36.0,979281502.0,2.0,...,2.0,-1.0,-1.0,52.550000,213987831.0,0.0,845825535.0,"[11, 55, 75, 93, 143, 151]",[0],0.0
2,8,0,151,0.2,0.0,1.386294,0.0,36.0,979281502.0,2.0,...,2.0,-1.0,-1.0,52.550000,213987831.0,0.0,845825535.0,"[11, 55, 75, 93, 143, 151]",[0],0.0
3,8,1,24,594.5,0.0,0.000000,0.0,120.0,56833659.0,1.0,...,2.0,213987831.0,562436811.0,52.550000,156242666.0,48040410.0,217178317.0,[24],[1],0.0
4,8,5,4,461.5,0.0,0.000000,0.0,37.0,979281502.0,1.0,...,1.0,213987831.0,-1.0,52.550000,600564501.0,979281502.0,-1.0,[4],[5],0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810978,1709396984676,73,23,20.3,0.0,0.000000,0.0,122.0,-1.0,1.0,...,1.0,-1.0,-1.0,25.545455,405077490.0,-1.0,-1.0,[23],[73],0.0
810979,1709396984676,74,68,46.7,0.0,0.000000,0.0,20.0,-1.0,2.0,...,2.0,485661706.0,-1.0,25.545455,485661706.0,338248086.0,-1.0,"[54, 68]",[74],0.0
810980,1709396984676,77,28,43.4,0.0,0.000000,0.0,16.0,926522633.0,1.0,...,1.0,253082810.0,-1.0,25.545455,209570018.0,588274547.0,112540362.0,[28],[77],0.0
810981,1709396984676,79,38,50.2,0.0,0.000000,0.0,16.0,269576388.0,2.0,...,1.0,253082810.0,-1.0,25.545455,313475193.0,269576388.0,-1.0,[38],"[6, 79]",0.0


In [107]:
from IPython.display import FileLink
subm.to_csv('/kaggle/working/subm.csv', index = False)
FileLink(r'subm.csv')

/kaggle/working/subm.csv